In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, roc_auc_score,log_loss, confusion_matrix, precision_score, recall_score, accuracy_score 
from sklearn import linear_model, ensemble , tree 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.linear_model import LogisticRegression
import imblearn.over_sampling
from sklearn.svm import SVC  
from sklearn.utils import class_weight
import statsmodels.api as sm
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline, make_pipeline 
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics

# データクレンジング 

・欠損値処理

In [2]:
train_data_df = pd.read_csv("credit_train.csv")
train_data_df

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train_data_df.shape

(100514, 19)

In [4]:
train_data_df.isnull().sum()

Loan ID                           514
Customer ID                       514
Loan Status                       514
Current Loan Amount               514
Term                              514
Credit Score                    19668
Annual Income                   19668
Years in current job             4736
Home Ownership                    514
Purpose                           514
Monthly Debt                      514
Years of Credit History           514
Months since last delinquent    53655
Number of Open Accounts           514
Number of Credit Problems         514
Current Credit Balance            514
Maximum Open Credit               516
Bankruptcies                      718
Tax Liens                         524
dtype: int64

In [5]:
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100514 entries, 0 to 100513
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       100000 non-null  object 
 1   Customer ID                   100000 non-null  object 
 2   Loan Status                   100000 non-null  object 
 3   Current Loan Amount           100000 non-null  float64
 4   Term                          100000 non-null  object 
 5   Credit Score                  80846 non-null   float64
 6   Annual Income                 80846 non-null   float64
 7   Years in current job          95778 non-null   object 
 8   Home Ownership                100000 non-null  object 
 9   Purpose                       100000 non-null  object 
 10  Monthly Debt                  100000 non-null  float64
 11  Years of Credit History       100000 non-null  float64
 12  Months since last delinquent  46859 non-null

In [6]:
repeat = train_data_df.duplicated()
print('repeat in train_data :', repeat.sum())

repeat in train_data : 10728


In [7]:
train_data_df.columns

Index(['Loan ID', 'Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')

In [8]:
train_data_df.columns = train_data_df.columns.str.replace(' ','_')
train_data_df.columns

Index(['Loan_ID', 'Customer_ID', 'Loan_Status', 'Current_Loan_Amount', 'Term',
       'Credit_Score', 'Annual_Income', 'Years_in_current_job',
       'Home_Ownership', 'Purpose', 'Monthly_Debt', 'Years_of_Credit_History',
       'Months_since_last_delinquent', 'Number_of_Open_Accounts',
       'Number_of_Credit_Problems', 'Current_Credit_Balance',
       'Maximum_Open_Credit', 'Bankruptcies', 'Tax_Liens'],
      dtype='object')

In [9]:
df_train, df_test  = train_test_split(train_data_df, test_size=0.2, random_state=42)
print('scale of train data:', df_train.shape)
print('scale of test data:', df_test.shape)

scale of train data: (80411, 19)
scale of test data: (20103, 19)


In [10]:
df_train = df_train.dropna(subset = ['Loan_ID'])
df_test = df_test.dropna(subset = ['Loan_ID'])
print('scale of train data:', df_train.shape)
print('scale of test data:', df_test.shape)

scale of train data: (79992, 19)
scale of test data: (20008, 19)


In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79992 entries, 62725 to 15795
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan_ID                       79992 non-null  object 
 1   Customer_ID                   79992 non-null  object 
 2   Loan_Status                   79992 non-null  object 
 3   Current_Loan_Amount           79992 non-null  float64
 4   Term                          79992 non-null  object 
 5   Credit_Score                  64536 non-null  float64
 6   Annual_Income                 64536 non-null  float64
 7   Years_in_current_job          76606 non-null  object 
 8   Home_Ownership                79992 non-null  object 
 9   Purpose                       79992 non-null  object 
 10  Monthly_Debt                  79992 non-null  float64
 11  Years_of_Credit_History       79992 non-null  float64
 12  Months_since_last_delinquent  37455 non-null  float64
 1

In [12]:
df_train.isnull().sum()

Loan_ID                             0
Customer_ID                         0
Loan_Status                         0
Current_Loan_Amount                 0
Term                                0
Credit_Score                    15456
Annual_Income                   15456
Years_in_current_job             3386
Home_Ownership                      0
Purpose                             0
Monthly_Debt                        0
Years_of_Credit_History             0
Months_since_last_delinquent    42537
Number_of_Open_Accounts             0
Number_of_Credit_Problems           0
Current_Credit_Balance              0
Maximum_Open_Credit                 2
Bankruptcies                      150
Tax_Liens                           5
dtype: int64

In [13]:
repeat_train_data = df_train.duplicated().sum()
repeat_test_data = df_test.duplicated().sum()
print('repeat data in train data', repeat_train_data)
print('repeat data in test data', repeat_test_data)

repeat data in train data 6520
repeat data in test data 401


In [14]:
df_train.drop_duplicates(inplace=True)
df_test.drop_duplicates(inplace=True)

repeat_train_data = df_train.duplicated().sum()
repeat_test_data = df_test.duplicated().sum()
print('repeat data in train data', repeat_train_data)
print('repeat data in test data', repeat_test_data)

print('scale of train data:', df_train.shape)
print('scale of test data:', df_test.shape)

repeat data in train data 0
repeat data in test data 0
scale of train data: (73472, 19)
scale of test data: (19607, 19)


In [15]:
df_train.isnull().sum()

Loan_ID                             0
Customer_ID                         0
Loan_Status                         0
Current_Loan_Amount                 0
Term                                0
Credit_Score                    15456
Annual_Income                   15456
Years_in_current_job             3117
Home_Ownership                      0
Purpose                             0
Monthly_Debt                        0
Years_of_Credit_History             0
Months_since_last_delinquent    39442
Number_of_Open_Accounts             0
Number_of_Credit_Problems           0
Current_Credit_Balance              0
Maximum_Open_Credit                 2
Bankruptcies                      143
Tax_Liens                           5
dtype: int64

In [16]:
df_train['Loan_ID'].value_counts().sort_values(ascending=False)

5a90cbe3-8fee-4582-8823-1f31546dec6e    2
770a1bf3-ebbb-435d-841a-02a654cf2f1f    2
b53fe85a-7866-47ef-82fa-745b2777377c    2
cecfe2d1-4019-421d-ab0c-78189195d96d    2
0d4b7e35-456d-4cd6-8dcf-dab2a192018e    2
                                       ..
17e5d465-4a54-4c83-bc1e-d53190ddf75b    1
cfc0577f-ad42-423d-ab31-6163c83a875c    1
af23ffc3-01c0-4b33-a63d-6b5d5018fade    1
bab59759-50cc-459b-b72f-1395281ea09b    1
b407196e-e336-4864-a2cf-218304285cb0    1
Name: Loan_ID, Length: 68478, dtype: int64

In [17]:
df_train[df_train['Loan_ID'] == '7830a00a-20c4-4480-9cf0-fe2f86b5266b']

,Loan_ID,Customer_ID,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
75405,7830a00a-20c4-4480-9cf0-fe2f86b5266b,f41798de-03d5-4c73-aa4d-1483774aa739,Fully Paid,99999999.0,Short Term,653.0,1314572.0,4 years,Rent,Debt Consolidation,13145.72,7.2,NaN,9.0,0.0,217132.0,323158.0,0.0,0.0


In [18]:
df_train[df_train['Loan_ID'] == '2b66ad68-5065-457c-a2ce-367bc48d8979']

,Loan_ID,Customer_ID,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
24682,2b66ad68-5065-457c-a2ce-367bc48d8979,71ec4684-1697-44eb-8222-a5a80c76dc0e,Fully Paid,346258.0,Short Term,704.0,1495224.0,9 years,Home Mortgage,Debt Consolidation,31399.78,25.5,67.0,16.0,0.0,539334.0,731236.0,0.0,0.0
7463,2b66ad68-5065-457c-a2ce-367bc48d8979,71ec4684-1697-44eb-8222-a5a80c76dc0e,Fully Paid,99999999.0,Short Term,704.0,1495224.0,9 years,Home Mortgage,Debt Consolidation,31399.78,25.5,67.0,16.0,0.0,539334.0,731236.0,0.0,0.0


In [19]:
df_train[(df_train.Loan_ID.duplicated() & (df_train['Current_Loan_Amount'] == 99999999.0))]

,Loan_ID,Customer_ID,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
48427,f001953b-a5e1-4668-8fc1-d8df1a9e72c0,702e5fcc-00bf-4722-8f3e-9b16b211e8de,Fully Paid,99999999.0,Long Term,688.0,674975.0,5 years,Rent,Business Loan,5495.37,14.7,45.0,12.0,0.0,157453.0,415316.0,0.0,0.0
37407,d3de2f28-3759-4453-a2d4-dd0f0dfed25f,b3110f03-0c03-45f5-9ca6-e368a1d39f40,Fully Paid,99999999.0,Long Term,696.0,851846.0,9 years,Own Home,Debt Consolidation,10647.98,9.5,NaN,8.0,0.0,129219.0,487366.0,0.0,0.0
67399,15a35601-a5b8-4f53-aaeb-e89bd9838f9b,a147b553-a0f1-4773-910d-9b1af0052324,Fully Paid,99999999.0,Long Term,667.0,1412897.0,10+ years,Rent,Debt Consolidation,36970.58,22.7,60.0,12.0,0.0,315248.0,651816.0,0.0,0.0
268,ea0e3c8d-0a6b-4b03-a766-c310c5a22cd6,e25e2a25-19db-4100-97fc-18282b8c98a0,Fully Paid,99999999.0,Short Term,721.0,805733.0,10+ years,Home Mortgage,Debt Consolidation,12757.55,15.9,45.0,9.0,0.0,93347.0,195448.0,0.0,0.0
31271,adebe0c9-c9d5-44fa-b70e-a827e2100415,2c2a199f-48b3-4b9b-91b4-e013dc45503c,Fully Paid,99999999.0,Long Term,678.0,1606279.0,< 1 year,Home Mortgage,Debt Consolidation,14858.00,29.5,23.0,20.0,1.0,203205.0,312884.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84896,cc537c28-c1ce-4b20-97fc-7947067688dd,e2293f56-2414-49dc-9bd7-5fbb2d64d385,Fully Paid,99999999.0,Long Term,722.0,1248186.0,8 years,Home Mortgage,Debt Consolidation,13729.97,17.5,NaN,8.0,0.0,526376.0,731698.0,0.0,0.0
52733,fc1deea3-945e-4ed4-acf7-71751b01797b,64577063-c0ec-4273-acab-e55105467aae,Fully Paid,99999999.0,Long Term,723.0,2412696.0,7 years,Home Mortgage,Debt Consolidation,12425.43,28.0,NaN,11.0,0.0,367897.0,777348.0,0.0,0.0
95003,35a73834-a7d1-47ae-9193-8f5da7f7f43b,a35a43a7-67dd-4989-a707-a389b6614beb,Fully Paid,99999999.0,Long Term,670.0,5225931.0,5 years,Rent,Debt Consolidation,18813.42,23.7,37.0,13.0,0.0,742596.0,1682692.0,0.0,0.0
35222,802d1d90-a269-4fdc-9697-1bec09c94128,a34e5841-7dad-4ec2-ac11-337cdc831782,Fully Paid,99999999.0,Short Term,683.0,843372.0,3 years,Home Mortgage,Debt Consolidation,13283.09,12.9,NaN,12.0,0.0,534508.0,724724.0,0.0,0.0


In [20]:
df_train = df_train[~(df_train.Loan_ID.duplicated() & (df_train['Current_Loan_Amount'] == 99999999.0))]
df_test = df_test[~(df_test.Loan_ID.duplicated() & (df_train['Current_Loan_Amount'] == 99999999.0))]

/Users/forrest/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [21]:
df_train[(df_train.Loan_ID.duplicated())]

,Loan_ID,Customer_ID,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
48159,ccb4bff4-4103-4220-bd6a-7cf8d97660f2,f7ad2ea9-c1a2-4020-8db9-d1ce98dd9cac,Fully Paid,215512.0,Short Term,716.0,1302868.0,7 years,Rent,Debt Consolidation,5830.34,13.8,NaN,6.0,0.0,84284.0,325270.0,0.0,0.0
72222,cb1970cb-57d1-4d98-9889-c95b36e4c72a,9a4fded3-a8a1-42e2-b10e-31ce3925e444,Fully Paid,357808.0,Long Term,671.0,1158810.0,2 years,Home Mortgage,Debt Consolidation,29260.00,12.5,1.0,27.0,0.0,331759.0,1401994.0,0.0,0.0
35114,6c36c90c-da96-42cf-b46d-81f50ec470f3,8a4d6d5b-69e0-4bde-afb2-cc1d3e8b831f,Fully Paid,456786.0,Long Term,700.0,1327606.0,1 year,Rent,Debt Consolidation,3186.30,15.2,51.0,5.0,0.0,105355.0,153626.0,0.0,0.0
50326,58d48443-93b1-4443-bd65-3ecd0fbbf9f7,d5d43933-bf2d-4b95-87e9-7b743568b618,Fully Paid,543290.0,Long Term,699.0,1370071.0,10+ years,Rent,Debt Consolidation,11017.72,14.1,NaN,8.0,0.0,340404.0,532620.0,0.0,0.0
2612,02287061-2f82-4870-a84f-7895ae1ad907,2a74c9b8-d631-46b4-aa2d-7b8b8b668034,Fully Paid,324148.0,Short Term,NaN,NaN,2 years,Own Home,Debt Consolidation,16480.98,5.7,NaN,14.0,0.0,22819.0,911284.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,abfdae7a-3bda-42db-ba79-4b9c059764bb,0a6c36c7-001d-4d98-9f35-26e0365228fa,Fully Paid,429088.0,Long Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,27129.34,12.3,NaN,13.0,0.0,351329.0,605352.0,0.0,0.0
67435,f35b054f-30c1-4e13-af01-685db773770f,3a42402c-3996-4021-a935-185eb1754fb1,Fully Paid,207086.0,Short Term,723.0,1645457.0,10+ years,Rent,other,34143.00,19.8,10.0,11.0,0.0,232218.0,321244.0,0.0,0.0
84654,47f27c67-3199-4aa4-a700-826e4a4b922c,d2b60744-df1a-4e25-a2f0-5ae12e6d31aa,Fully Paid,546260.0,Long Term,725.0,1698372.0,10+ years,Rent,Debt Consolidation,21512.75,17.5,12.0,19.0,0.0,88787.0,1606352.0,0.0,0.0
53707,5f51b043-e1cb-4022-8085-1aed0142d5f0,8a187950-2cc1-470b-852e-d32e13cb49af,Fully Paid,99242.0,Short Term,703.0,666596.0,3 years,Own Home,Debt Consolidation,9110.12,9.1,18.0,9.0,0.0,126882.0,282546.0,0.0,0.0


In [22]:
df_train[df_train['Loan_ID'] == 'ccb4bff4-4103-4220-bd6a-7cf8d97660f2']

,Loan_ID,Customer_ID,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
18178,ccb4bff4-4103-4220-bd6a-7cf8d97660f2,f7ad2ea9-c1a2-4020-8db9-d1ce98dd9cac,Fully Paid,215512.0,Short Term,NaN,NaN,7 years,Rent,Debt Consolidation,5830.34,13.8,NaN,6.0,0.0,84284.0,325270.0,0.0,0.0
48159,ccb4bff4-4103-4220-bd6a-7cf8d97660f2,f7ad2ea9-c1a2-4020-8db9-d1ce98dd9cac,Fully Paid,215512.0,Short Term,716.0,1302868.0,7 years,Rent,Debt Consolidation,5830.34,13.8,NaN,6.0,0.0,84284.0,325270.0,0.0,0.0


In [23]:
df_train = df_train.dropna(subset = ['Annual_Income'])
df_test = df_test.dropna(subset = ['Annual_Income'])

In [24]:
df_train.Purpose.unique()

array(['Debt Consolidation', 'Buy House', 'Home Improvements', 'other',
       'Other', 'Business Loan', 'small_business', 'Take a Trip',
       'Buy a Car', 'Medical Bills', 'major_purchase', 'moving',
       'wedding', 'Educational Expenses', 'renewable_energy', 'vacation'],
      dtype=object)

In [25]:
df_train.Purpose = df_train.Purpose.str.replace('other','Other')
df_test.Purpose = df_test.Purpose.str.replace('other','Other')

In [26]:
df_train.Purpose.unique()

array(['Debt Consolidation', 'Buy House', 'Home Improvements', 'Other',
       'Business Loan', 'small_business', 'Take a Trip', 'Buy a Car',
       'Medical Bills', 'major_purchase', 'moving', 'wedding',
       'Educational Expenses', 'renewable_energy', 'vacation'],
      dtype=object)

In [27]:
df_train.isnull().sum()

Loan_ID                             0
Customer_ID                         0
Loan_Status                         0
Current_Loan_Amount                 0
Term                                0
Credit_Score                        0
Annual_Income                       0
Years_in_current_job             2416
Home_Ownership                      0
Purpose                             0
Monthly_Debt                        0
Years_of_Credit_History             0
Months_since_last_delinquent    30431
Number_of_Open_Accounts             0
Number_of_Credit_Problems           0
Current_Credit_Balance              0
Maximum_Open_Credit                 1
Bankruptcies                      112
Tax_Liens                           3
dtype: int64

In [28]:
df_train = df_train.drop_duplicates(subset = ['Loan_ID'])
df_test = df_test.drop_duplicates(subset = ['Loan_ID'])
df_train.isnull().sum()

Loan_ID                             0
Customer_ID                         0
Loan_Status                         0
Current_Loan_Amount                 0
Term                                0
Credit_Score                        0
Annual_Income                       0
Years_in_current_job             2368
Home_Ownership                      0
Purpose                             0
Monthly_Debt                        0
Years_of_Credit_History             0
Months_since_last_delinquent    29853
Number_of_Open_Accounts             0
Number_of_Credit_Problems           0
Current_Credit_Balance              0
Maximum_Open_Credit                 1
Bankruptcies                      109
Tax_Liens                           3
dtype: int64

In [29]:
df_train['Years_in_current_job'].value_counts()

10+ years    17404
2 years       5070
3 years       4582
< 1 year      4518
5 years       3850
1 year        3579
4 years       3379
6 years       3165
7 years       3097
8 years       2563
9 years       2143
Name: Years_in_current_job, dtype: int64

In [30]:
df_train['Years_in_current_job'] = df_train['Years_in_current_job'].fillna('10+ years')
df_test['Years_in_current_job'] = df_test['Years_in_current_job'].fillna('10+ years')
df_train.isnull().sum()

Loan_ID                             0
Customer_ID                         0
Loan_Status                         0
Current_Loan_Amount                 0
Term                                0
Credit_Score                        0
Annual_Income                       0
Years_in_current_job                0
Home_Ownership                      0
Purpose                             0
Monthly_Debt                        0
Years_of_Credit_History             0
Months_since_last_delinquent    29853
Number_of_Open_Accounts             0
Number_of_Credit_Problems           0
Current_Credit_Balance              0
Maximum_Open_Credit                 1
Bankruptcies                      109
Tax_Liens                           3
dtype: int64

In [31]:
df_train = df_train.drop(columns='Months_since_last_delinquent')
df_test = df_test.drop(columns='Months_since_last_delinquent')

In [32]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [33]:
df_train.isnull().sum()

Loan_ID                      0
Customer_ID                  0
Loan_Status                  0
Current_Loan_Amount          0
Term                         0
Credit_Score                 0
Annual_Income                0
Years_in_current_job         0
Home_Ownership               0
Purpose                      0
Monthly_Debt                 0
Years_of_Credit_History      0
Number_of_Open_Accounts      0
Number_of_Credit_Problems    0
Current_Credit_Balance       0
Maximum_Open_Credit          0
Bankruptcies                 0
Tax_Liens                    0
dtype: int64

In [34]:
df_train.duplicated().sum()

0

In [35]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55608 entries, 62725 to 860
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Loan_ID                    55608 non-null  object 
 1   Customer_ID                55608 non-null  object 
 2   Loan_Status                55608 non-null  object 
 3   Current_Loan_Amount        55608 non-null  float64
 4   Term                       55608 non-null  object 
 5   Credit_Score               55608 non-null  float64
 6   Annual_Income              55608 non-null  float64
 7   Years_in_current_job       55608 non-null  object 
 8   Home_Ownership             55608 non-null  object 
 9   Purpose                    55608 non-null  object 
 10  Monthly_Debt               55608 non-null  float64
 11  Years_of_Credit_History    55608 non-null  float64
 12  Number_of_Open_Accounts    55608 non-null  float64
 13  Number_of_Credit_Problems  55608 non-null  f

In [36]:
bank_lone_train = pd.get_dummies(df_train, columns =['Term','Home_Ownership','Purpose','Loan_Status', 'Years_in_current_job'], drop_first=True)
bank_lone_test = pd.get_dummies(df_test, columns =['Term','Home_Ownership','Purpose','Loan_Status', 'Years_in_current_job'], drop_first=True)

In [37]:
bank_lone_train.columns

Index(['Loan_ID', 'Customer_ID', 'Current_Loan_Amount', 'Credit_Score',
       'Annual_Income', 'Monthly_Debt', 'Years_of_Credit_History',
       'Number_of_Open_Accounts', 'Number_of_Credit_Problems',
       'Current_Credit_Balance', 'Maximum_Open_Credit', 'Bankruptcies',
       'Tax_Liens', 'Term_Short Term', 'Home_Ownership_Home Mortgage',
       'Home_Ownership_Own Home', 'Home_Ownership_Rent', 'Purpose_Buy House',
       'Purpose_Buy a Car', 'Purpose_Debt Consolidation',
       'Purpose_Educational Expenses', 'Purpose_Home Improvements',
       'Purpose_Medical Bills', 'Purpose_Other', 'Purpose_Take a Trip',
       'Purpose_major_purchase', 'Purpose_moving', 'Purpose_renewable_energy',
       'Purpose_small_business', 'Purpose_vacation', 'Purpose_wedding',
       'Loan_Status_Fully Paid', 'Years_in_current_job_10+ years',
       'Years_in_current_job_2 years', 'Years_in_current_job_3 years',
       'Years_in_current_job_4 years', 'Years_in_current_job_5 years',
       'Years_in_cu

In [38]:
df_train.corr()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
Current_Loan_Amount,1.000000,-0.105117,0.015514,-0.008151,0.026922,0.001346,-0.004906,0.006711,-0.000827,-0.001934,-0.002628
Credit_Score,-0.105117,1.000000,-0.014864,-0.000472,-0.013054,0.006577,0.000285,-0.003221,-0.003941,-0.003473,0.006607
Annual_Income,0.015514,-0.014864,1.000000,0.459476,0.148069,0.137162,-0.014591,0.304946,0.042415,-0.045649,0.040754
Monthly_Debt,-0.008151,-0.000472,0.459476,1.000000,0.194691,0.407775,-0.051059,0.484890,0.041829,-0.076626,0.021754
Years_of_Credit_History,0.026922,-0.013054,0.148069,0.194691,1.000000,0.131483,0.058908,0.210841,0.029104,0.061631,0.017174
Number_of_Open_Accounts,0.001346,0.006577,0.137162,0.407775,0.131483,1.000000,-0.014407,0.234957,0.033140,-0.025860,0.007073
Number_of_Credit_Problems,-0.004906,0.000285,-0.014591,-0.051059,0.058908,-0.014407,1.000000,-0.110801,-0.011297,0.749409,0.589154
Current_Credit_Balance,0.006711,-0.003221,0.304946,0.484890,0.210841,0.234957,-0.110801,1.000000,0.136300,-0.124809,-0.011894
Maximum_Open_Credit,-0.000827,-0.003941,0.042415,0.041829,0.029104,0.033140,-0.011297,0.136300,1.000000,-0.015012,-0.000835
Bankruptcies,-0.001934,-0.003473,-0.045649,-0.076626,0.061631,-0.025860,0.749409,-0.124809,-0.015012,1.000000,0.041065


In [39]:
plt.figure(figsize=(10,8))
data_corr = df_train.corr()
mask = np.triu(np.ones_like(data_corr, dtype=np.bool))
mask = mask[1:, :-1]
corr = data_corr.iloc[1:,:-1].copy()
sns.heatmap(corr, cmap = 'pink_r', annot = True, vmin= -1, vmax= 1, linewidths=1.5, fmt='.2f', mask=mask);
plt.title('CORRELATION BETWEEN FEATURES\n', loc='left', fontsize=18);

In [40]:
bank_lone_train = bank_lone_train[bank_lone_train['Current_Loan_Amount'] != 99999999]
bank_lone_train = bank_lone_train[((bank_lone_train['Current_Loan_Amount'] <= 600000 )
                                   & (bank_lone_train['Term_Short Term']==1))
                                  | (bank_lone_train['Term_Short Term']==0)]

bank_lone_test = bank_lone_test[bank_lone_test['Current_Loan_Amount'] != 99999999]
bank_lone_test = bank_lone_test[((bank_lone_test['Current_Loan_Amount'] <= 600000 )
                                   & (bank_lone_test['Term_Short Term']==1))
                                  | (bank_lone_test['Term_Short Term']==0)]

In [41]:
bank_lone_train = bank_lone_train.loc[bank_lone_train['Credit_Score'] <= 1500,:]
bank_lone_train = bank_lone_train.loc[bank_lone_train['Credit_Score'] >= 620 ,:]
bank_lone_train = bank_lone_train[((bank_lone_train['Credit_Score'] >= 680 )
                                   & (bank_lone_train['Term_Short Term']==1))| 
                                  (bank_lone_train['Term_Short Term']==0)]

bank_lone_test = bank_lone_test.loc[bank_lone_test['Credit_Score'] <= 1500,:]
bank_lone_test = bank_lone_test.loc[bank_lone_test['Credit_Score'] >= 620 ,:]
bank_lone_test = bank_lone_test[((bank_lone_test['Credit_Score'] >= 680 )
                                   & (bank_lone_test['Term_Short Term']==1))| 
                                  (bank_lone_test['Term_Short Term']==0)]

In [42]:
bank_lone_train = bank_lone_train.loc[bank_lone_train['Annual_Income'] <= 2750000,:]
bank_lone_train = bank_lone_train[((bank_lone_train['Annual_Income'] <= 2395000 )
                                   & (bank_lone_train['Term_Short Term']==1))
                                  | (bank_lone_train['Term_Short Term']==0)]

bank_lone_test = bank_lone_test.loc[bank_lone_test['Annual_Income'] <= 2750000,:]
bank_lone_test = bank_lone_test[((bank_lone_test['Annual_Income'] <= 2395000 )
                                   & (bank_lone_test['Term_Short Term']==1))
                                  | (bank_lone_test['Term_Short Term']==0)]

In [43]:
bank_lone_train = bank_lone_train.loc[bank_lone_train['Monthly_Debt'] <= 44500,:]
bank_lone_train = bank_lone_train[((bank_lone_train['Monthly_Debt'] <= 36000 )& (bank_lone_train['Term_Short Term']==1))| 
                                  (bank_lone_train['Term_Short Term']==0)]

bank_lone_test = bank_lone_test.loc[bank_lone_test['Monthly_Debt'] <= 44500,:]
bank_lone_test = bank_lone_test[((bank_lone_test['Monthly_Debt'] <= 36000 )& 
                                   (bank_lone_test['Term_Short Term']==1))| 
                                  (bank_lone_test['Term_Short Term']==0)]

In [44]:
bank_lone_train = bank_lone_train.loc[bank_lone_train['Current_Credit_Balance'] <= 760000,:]
bank_lone_train = bank_lone_train[((bank_lone_train['Current_Credit_Balance'] <= 504000 )& 
                                   (bank_lone_train['Term_Short Term']==1))| (bank_lone_train['Term_Short Term']==0)]

bank_lone_test = bank_lone_test.loc[bank_lone_test['Current_Credit_Balance'] <= 760000,:]
bank_lone_test = bank_lone_test[((bank_lone_test['Current_Credit_Balance'] <= 504000 )& 
                                   (bank_lone_test['Term_Short Term']==1))| 
                                  (bank_lone_test['Term_Short Term']==0)]

In [45]:
bank_lone_train = bank_lone_train.loc[bank_lone_train['Maximum_Open_Credit'] <= 1400000,:]
bank_lone_train = bank_lone_train[((bank_lone_train['Maximum_Open_Credit'] <= 990000 )& 
                                   (bank_lone_train['Term_Short Term']==1))| (bank_lone_train['Term_Short Term']==0)]

bank_lone_test = bank_lone_test.loc[bank_lone_test['Maximum_Open_Credit'] <= 1400000,:]
bank_lone_test = bank_lone_test[((bank_lone_test['Maximum_Open_Credit'] <= 990000 )& 
                                   (bank_lone_test['Term_Short Term']==1))| 
                                  (bank_lone_test['Term_Short Term']==0)]

In [46]:
X_train = bank_lone_train.drop(['Term_Short Term','Loan_ID','Customer_ID',
                                'Credit_Score', 'Years_of_Credit_History', 
                                'Number_of_Credit_Problems', 'Number_of_Open_Accounts',
                                'Bankruptcies'], axis = 1)
y_train = bank_lone_train['Term_Short Term']
X_test = bank_lone_test.drop(['Term_Short Term','Loan_ID','Customer_ID',
                                'Credit_Score', 'Years_of_Credit_History', 
                                'Number_of_Credit_Problems', 'Number_of_Open_Accounts',
                                'Bankruptcies'], axis = 1)
y_test = bank_lone_test['Term_Short Term']

In [47]:
model = sm.OLS(y_train,X_train)
fit = model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:        Term_Short Term   R-squared (uncentered):                 -48.324
Model:                            OLS   Adj. R-squared (uncentered):            -48.376
Method:                 Least Squares   F-statistic:                             -941.7
Date:                Tue, 17 May 2022   Prob (F-statistic):                        1.00
Time:                        10:18:50   Log-Likelihood:                         -14633.
No. Observations:               32713   AIC:                                  2.933e+04
Df Residuals:                   32679   BIC:                                  2.962e+04
Df Model:                          34                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Current_Loan_Amount            -1.545e-06   1.69e-08    -91.578      0.000   -1.58e-06   -1.51e-06
Annual_Income                   5.935e-08   5.65e-09     10.500      0.000    4.83e-08    7.04e-08
Monthly_Debt                   -2.882e-06   3.19e-07     -9.036      0.000   -3.51e-06   -2.26e-06
Current_Credit_Balance         -5.766e-08   2.36e-08     -2.448      0.014   -1.04e-07   -1.15e-08
Maximum_Open_Credit            -7.519e-08   1.19e-08     -6.311      0.000   -9.85e-08   -5.18e-08
Tax_Liens                          0.0096      0.008      1.174      0.241      -0.006       0.026
Home_Ownership_Home Mortgage       0.9325      0.020     45.955      0.000       0.893       0.972
Home_Ownership_Own Home            0.9586      0.021     45.944      0.000       0.918       0.999
Home_Ownership_Rent                0.9762      0.020     49.223      0.000       0.937       1.015
Purpose_Buy House                  0.0469      0.032      1.455      0.146      -0.016       0.110
Purpose_Buy a Car                 -0.0315      0.025     -1.237      0.216      -0.081       0.018
Purpose_Debt Consolidation         0.1189      0.018      6.683      0.000       0.084       0.154
Purpose_Educational Expenses      -0.0339      0.070     -0.483      0.629      -0.172       0.104
Purpose_Home Improvements          0.0575      0.020      2.896      0.004       0.019       0.096
Purpose_Medical Bills              0.0450      0.027      1.651      0.099      -0.008       0.098
Purpose_Other                      0.0415      0.019      2.193      0.028       0.004       0.079
Purpose_Take a Trip                0.3823      0.031     12.244      0.000       0.321       0.444
Purpose_major_purchase            -0.0168      0.039     -0.427      0.670      -0.094       0.060
Purpose_moving                     0.0414      0.066      0.631      0.528      -0.087       0.170
Purpose_renewable_energy           0.0514      0.268      0.191      0.848      -0.475       0.577
Purpose_small_business             0.0546      0.047      1.166      0.244      -0.037       0.147
Purpose_vacation                   0.0729      0.078      0.937      0.349      -0.080       0.225
Purpose_wedding                    0.1790      0.065      2.767      0.006       0.052       0.306
Loan_Status_Fully Paid             0.0886      0.005     17.738      0.000       0.079       0.098
Years_in_current_job_10+ years     0.0197      0.009      2.194      0.028       0.002       0.037
Years_in_current_job_2 years       0.0262      0.011      2.473      0.013       0.005       0.047
Years_in_current_job_3 years       0.0244      0.011      2.258      0.024       0.003       0.046
Years_in_curre

In [48]:
y_train.value_counts()

1    22879
0     9834
Name: Term_Short Term, dtype: int64

In [49]:
long_term_0 = bank_lone_train[bank_lone_train['Term_Short Term'] == 0]
short_term_1 = bank_lone_train[bank_lone_train['Term_Short Term'] == 1]
print('Long term 0:', long_term_0.shape[0])
print('Short term 1:', short_term_1.shape[0])

Long term 0: 9834
Short term 1: 22879


# モデリング

・ロジスティック

In [50]:
params = {'C': [0.001,0.01,0.1,1,10,100,1000], 'class_weight':[{0:0,0:0.01,0:0.1,0:0.5,0:1,0:10,0:2} ] }
lr_grid = GridSearchCV(LogisticRegression(), param_grid = params, scoring='f1', cv = 5)
lr_grid.fit(X_train, y_train)
print('\n Best param after grid search: ', lr_grid.best_params_ )
print(' Best f1_score for cross validation: ',lr_grid.best_score_ )


 Best param after grid search:  {'C': 0.001, 'class_weight': {0: 2}}
 Best f1_score for cross validation:  0.7170806194365902


In [51]:
LR = LogisticRegression(C= 0.001 ,solver='liblinear')
kf = KFold(n_splits=10, random_state=42, shuffle=True)
cr_f1 = cross_val_score(LR, X_train, y_train, scoring='f1', cv=kf)
print('\n Normal Logistic Regression Valdition F1: \n',cr_f1)
print('\n Mean Normal Logistic Regression Valdition F1: \n',cr_f1.mean())
print('--------------------------------')


 Normal Logistic Regression Valdition F1: 
 [0.80033913 0.8061049  0.80355209 0.81094844 0.80573385 0.81843397
 0.81005587 0.80310211 0.79921943 0.80628498]

 Mean Normal Logistic Regression Valdition F1: 
 0.8063774752538592
--------------------------------


In [52]:
lr_balanced = LogisticRegression(class_weight='balanced', solver='liblinear')
cr_balnced_f1 = cross_val_score(lr_balanced, X_train, y_train, scoring='f1', cv=kf)
print('\n Balanced class weights Logistic Regression Valdition F1: \n',cr_balnced_f1)
print('\n Mean Balanced class weights Logistic Regression Valdition F1: \n',cr_balnced_f1.mean())
print('--------------------------------')



 Balanced class weights Logistic Regression Valdition F1: 
 [0.69856704 0.68117866 0.67982693 0.69860896 0.67910644 0.6945388
 0.701063   0.67845829 0.68236553 0.68821192]

 Mean Balanced class weights Logistic Regression Valdition F1: 
 0.6881925584835079
--------------------------------


In [53]:
lr_4x = LogisticRegression(C= 0.001, class_weight={0 : 2, 1 : 1}, solver='liblinear')
cr_weghts_f1 = cross_val_score(lr_4x, X_train, y_train, scoring='f1', cv=kf)
print('\n Class weights Logistic Regression Valdition F1: \n',cr_weghts_f1)
print('\n Mean Class weights Logistic Regression Valdition F1: \n',cr_weghts_f1.mean())
print('--------------------------------')


 Class weights Logistic Regression Valdition F1: 
 [0.72161895 0.71736375 0.71184687 0.72370057 0.70962851 0.72622334
 0.72863568 0.70393058 0.71534717 0.71443193]

 Mean Class weights Logistic Regression Valdition F1: 
 0.71727273536161
--------------------------------


In [54]:
imba_pipeline = make_pipeline(SMOTE(random_state=42), 
                              LogisticRegression(C= 0.001, solver='liblinear'))

imba_val = cross_val_score(imba_pipeline, X_train, y_train, scoring='f1', cv=kf)
print('\n Smote Logistic Regression Valdition F1: \n', imba_val)
print('\n Mean Smote Logistic Regression Valdition F1: \n', imba_val.mean())


 Smote Logistic Regression Valdition F1: 
 [0.69954011 0.68291391 0.68320302 0.70020534 0.68082874 0.69908973
 0.70289855 0.67895707 0.6846318  0.68903567]

 Mean Smote Logistic Regression Valdition F1: 
 0.6901303941714383


In [55]:
LR.fit(X_train, y_train)
y_predict = (LR.predict_proba(X_train)[:, 1] >= 0.65)

print("Default threshold:")
print("Precision: {:6.4f},   Recall: {:6.4f}".format(precision_score(y_train, y_predict), 
                                                     recall_score(y_train, y_predict)))

Default threshold:
Precision: 0.8209,   Recall: 0.5669


In [56]:
y_predict = (LR.predict_proba(X_train)[:, 1] >= 0.624)
loan_confusion = confusion_matrix(y_train, y_predict)
sns.heatmap(loan_confusion , cmap = 'pink_r', annot = True , square = True , fmt = 'd',
           xticklabels = ['long term','short term'],
           yticklabels = ['long term','short term'])
plt.title('Train Confusion Matrix',fontsize = 15)
plt.xlabel('prediction')
plt.ylabel('actual');

・決定木

In [57]:
Decision_Tree = DecisionTreeClassifier(max_depth = 8)
Decision_Tree.fit(X_train, y_train)
scores = cross_val_score(Decision_Tree, X_train, y_train, cv=5, scoring='f1')

In [58]:
dt_bal = DecisionTreeClassifier(max_depth = 8, class_weight='balanced')
dt_bal.fit(X_train, y_train)
scores = cross_val_score(dt_bal, X_train, y_train, cv=5, scoring='f1')

In [59]:
dt_wtd = DecisionTreeClassifier(class_weight= {0 : 10, 1 : 1})
scores = cross_val_score(dt_wtd, X_train, y_train, cv=5, scoring='f1')
dt_wtd.fit(X_train, y_train)

DecisionTreeClassifier(class_weight={0: 10, 1: 1})

In [63]:
tree_param = {'criterion':['gini','entropy'],
              'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}

gd_sr = GridSearchCV(DecisionTreeClassifier(), param_grid=tree_param,scoring='f1',cv=5,n_jobs=-1)
gd_sr.fit(X_train, y_train)
best_parameters = gd_sr.best_params_
print('Normal Decision Tree Valdition F1:',scores.mean())
print('Balanced class weights Decision Tree Valdition F1:',scores.mean())
print('10:1 class weights Decision Tree Valdition F1:',scores.mean())
print('\n Best param after grid search', best_parameters)
print('\n Best score after grid search', gd_sr.best_score_)

Normal Decision Tree Valdition F1: 0.8188182126297082
Balanced class weights Decision Tree Valdition F1: 0.8188182126297082
10:1 class weights Decision Tree Valdition F1: 0.8188182126297082

 Best param after grid search {'criterion': 'entropy', 'max_depth': 8}

 Best score after grid search 0.8702528429069607


In [64]:
y_predict = (Decision_Tree.predict_proba(X_train)[:, 1] >= 0.5569)

print("Default threshold:")
print("Precision: {:6.4f},   Recall: {:6.4f}".format(precision_score(y_train, y_predict), 
                                                     recall_score(y_train, y_predict)))

Default threshold:
Precision: 0.8434,   Recall: 0.8825


・ランダムフォレスト

In [65]:
Random_Forest = RandomForestClassifier(n_estimators = 5, random_state=1)
Random_Forest.fit(X_train, y_train)
scores = cross_val_score(Random_Forest, X_train, y_train, cv=10, scoring='f1')

In [66]:
rf_bal = RandomForestClassifier(n_estimators = 10, random_state=1, class_weight='balanced')
rf_bal.fit(X_train, y_train)
scores = cross_val_score(rf_bal, X_train, y_train, cv=10, scoring='f1')

In [68]:
rf_wtd = RandomForestClassifier(n_estimators = 10, random_state=1, class_weight= {0 : 2, 1 : 1})
rf_wtd.fit(X_train, y_train)
scores = cross_val_score(rf_wtd, X_train, y_train, cv=10, scoring='f1')
print('\n Mean Normal Random Forest Valdition F1:  \n',scores.mean())
print('\n Mean Balanced class weights Random Forest Valdition F1: \n',scores.mean())
print('\n 2:1 class weights Random Forest Valdition F1: \n',scores.mean())


 Mean Normal Random Forest Valdition F1:  
 0.8467494887782726

 Mean Balanced class weights Random Forest Valdition F1: 
 0.8467494887782726

 2:1 class weights Random Forest Valdition F1: 
 0.8467494887782726


In [70]:
y_predict = (Random_Forest.predict_proba(X_train)[:, 1] >= 0.66)
print("Default threshold:")
print("Precision: {:6.4f},   Recall: {:6.4f}".format(precision_score(y_train, y_predict), 
                                                     recall_score(y_train, y_predict)))

Default threshold:
Precision: 0.9961,   Recall: 0.9338


・XGBoost

In [71]:
X_train.columns = X_train.columns.str.replace('<','less').str.replace(' ','_')

xgboost = XGBClassifier(n_estimators = 100, learning_rate = 0.05)
xgboost.fit(X_train, y_train)
accuracies = cross_val_score(estimator = xgboost, X = X_train, y = y_train, cv = 5, scoring='f1')

/Users/forrest/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:41:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/forrest/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:41:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/forrest/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[12:41:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/forrest/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:41:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/forrest/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:41:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [72]:
print('Mean f1_score XGBoost for cross validation : ',accuracies.mean())

Mean f1_score XGBoost for cross validation :  0.873151250557083


In [73]:
y_predict = (xgboost.predict_proba(X_train)[:, 1] >= 0.1)

print("Default threshold:")
print("Precision: {:6.4f},   Recall: {:6.4f}".format(precision_score(y_train, y_predict), 
                                                     recall_score(y_train, y_predict)))

Default threshold:
Precision: 0.7634,   Recall: 1.0000


# テスト

In [74]:
y_pred_ =xgboost.predict(X_test)
y_pred_1 =xgboost.predict(X_train)
scores_1 = metrics.f1_score(y_train, y_pred_1)
scores = metrics.f1_score(y_test, y_pred_)
print('Test score for XGBoost: ', scores)

Test score for XGBoost:  0.8691032724181896
